In [ ]:
#|default_exp dhb

In [ ]:
#| export
#|export
doc = """**Backup Chat for SolveIt using dialoghelper and lisette**

Sometimes we may have a problem in SolveIt while Sonnet is down (E300), or maybe we want a different perspective.

This module helps us to leverage any other LLM that is available to LiteLLM by providing our own keys and the model name.

Usage: 
```python
from solveit_dmtools import dhb

# then in another cell
# bc = dhb.c() to search model names
bc = dhb.c("model-name")
bc("Hi")
```
"""
import json
import re
from dialoghelper.core import *
from lisette import *
from typing import Optional, Union
from ipykernel_helper import read_url
# from fastcore.utils import patch

class BackupChat(Chat):
    models = None
    vars_for_hist = None
    model = None

    def __init__(self,
                model: str = None,
                sp='',
                temp=0,
                search=False,
                tools: list = None,
                hist: list = None,
                ns: Optional[dict] = None,
                cache=False,
                cache_idxs: list = [-1],
                ttl=None,
                var_names: list = None,
    ):
        if sp is None or sp == '':
            sp = """You're continuing a conversation from another session. Variables are marked as $`varname` and tools as &`toolname` in the context.

If you see references to variables or tools that might be relevant to your answer but aren't fully available, ask the user to indicate which ones they want to include by calling e.g their `bc.add_vars`, `bc.add_tools`, or `bc.add_vars_and_tools` methods (if they called their chat instance `bc`). Note that these 3 methods each take a list of names or a string containing space-delimited names.

Tool results from the earlier conversation may be truncated to about 100 characters. If you need complete information, you should ask the user to run the tool and store results in a variable then make that variable available using the chat object's add_vars method. You already have access to the read_url tool, but do confirm if you can read the URLs once because it may be expensive to access them.

You are not able to run other code so you cannot store your own variables or do that for me, instead you should give Python in fenced markdown in your responses. If giving code examples or similar, remember to use fenced markdown too.

Use a Socratic approach - guide through questions rather than direct answers - unless the user explicitly asks you to do something differently."""
        if self.models is None:
            self.models = self.get_litellm_models()

        if model is None:
            _m1 = input("Please enter part of a model name to pick your model. Remember you also need to have secret for their API key already defined in your secrets:")
            print("Please try again by using e.g. `bc = dhb.c('model_name')` with a model name in:")
            print('\n'.join([m for m in self.models if _m1 in m]))
            return None
        if model not in self.models:
            raise ValueError(f"Model {model} not found in LiteLLM models. Please check the model name or use a different model.")
        self.model = model
        self.vars_for_hist = dict()
        if var_names is not None:
            self.add_vars(var_names)
        if tools is None:
            tools = [read_url]
        super().__init__(model=model, sp=sp, temp=temp, search=search, tools=tools, hist=hist, ns=ns, cache=cache, cache_idxs=cache_idxs, ttl=ttl)

    def get_litellm_models(self):
        url = "https://raw.githubusercontent.com/BerriAI/litellm/refs/heads/main/model_prices_and_context_window.json"
        data = read_url(url, as_md=False)
        models = json.loads(data)
        return [k for k in models.keys() if k != 'sample_spec']
    
    def __call__(self, 
                msg=None,
                prefill=None,
                temp=None,
                think=None,
                search=None,
                stream=False,
                max_steps=2,
                final_prompt='You have no more tool uses. Please summarize your findings. If you did not complete your goal please tell the user what further work needs to be done so they can choose how best to proceed.',
                return_all=False,
                var_names=None, # list of variable names to add to the chat
                **kwargs,
                ):
        msgs = [m for m in find_msgs() if m['pinned'] or not m['skipped']]
        curr_msg = read_msg(0)['msg']
        if var_names: self.add_vars(var_names)
        self.hist = self._build_hist(msgs, msgid=curr_msg['id'])
        start = len(self.hist)
        update_msg(msgid=curr_msg['id'], content="# " + curr_msg['content'].replace('\n', '\n# '), i_collapsed=1, o_collapsed=1)
        response = super().__call__(msg=msg, prefill=prefill, temp=temp, think=think, search=search, stream=stream, max_steps=max_steps, final_prompt=final_prompt, return_all=return_all, **kwargs)
        output = self._new_msgs_to_output(start)
        add_msg(content=f"**Prompt ({self.model}):** {msg}", output=output, msg_type='prompt')
        return response

    def _build_hist(self, msgs:list, msgid:str=None):
        if msgid is None: curr = len(msgs)
        else:
            try: curr = next(i for i,m in enumerate(msgs) if m['id'] == msgid)
            except StopIteration: curr = len(msgs)
        hist = []
        for m in msgs[:curr]:
            eol = '\n'
            if m['msg_type'] == 'code': hist.append({'role': 'user', 'content': f"```python{eol}{m['content']}{eol}```{eol}Output: {m.get('output', '[]')}"})
            elif m['msg_type'] == 'note': hist.append({'role': 'user', 'content': m['content']})
            elif m['msg_type'] == 'prompt':
                hist.append({'role': 'user', 'content': m['content']})
                if m.get('output'): hist.append({'role': 'assistant', 'content': m['output']})
        
        hist = mk_msgs(hist + self._vars_as_msg())
        return hist

    def _vars_as_msg(self):
        if self.vars_for_hist and len(self.vars_for_hist):
            content = "Here are the requested variables:\n" + "\n".join([f"$`{v.strip()}`: {globals().get(v.strip(), 'NOT FOUND')}" for v in self.vars_for_hist.keys()])
            return [{'role': 'user', 'content': content}]
        else:
            return []

    def _new_msgs_to_output(self, start):
        new_msgs = self.hist[start+1:]
        parts = []
        for i, m in enumerate(new_msgs):
            if m.get('role') == 'assistant' and m.get('tool_calls'):
                for tc in m['tool_calls']:
                    result_msg = next((r for r in new_msgs if r.get('tool_call_id') == tc['id']), None)
                    if result_msg: parts.append(self._format_tool_details(tc['id'], tc['function']['name'], json.loads(tc['function']['arguments']), result_msg['content'], is_last_msg=(i == len(new_msgs)-1)))
            elif m.get('role') == 'assistant' and m.get('content'):
                content = m['content']
                if 'You have no more tool uses' not in content: parts.append(content)
        return '\n\n'.join(parts)
    
    def _trunc_tool_result(self, result, max_len=100, is_last_msg=False):
        if len(str(result)) <= max_len: return result
        return str(result)[:max_len] + '<TRUNCATED>'
    
    def _format_tool_details(self, tool_id, func_name, args, result, is_last_msg=False):
        result_str = self._trunc_tool_result(result)
        tool_json = json.dumps({"id": tool_id, "call": {"function": func_name, "arguments": args}, "result": result_str}, indent=2)
        return f"<details class='tool-usage-details'>\n\n```json\n{tool_json}\n```\n\n</details>"    
    
    def add_vars(self, var_names:Union[list,str]=None):
        "Add variables to conversation as user message"
        if isinstance(var_names, str):
            var_names = var_names.split()
        if not isinstance(var_names, list):
            raise ValueError(f"var_names must be a string or list of strings, not {type(var_names)}")
        
        # Add each var to the self.vars_for_hist dictionary
        for v in var_names:
            self.vars_for_hist[v.strip()] = globals().get(v.strip(), 'NOT FOUND')
    
    def add_tools(self, tool_names:Union[list,str]=None):
        "Add tools to the chat's tool list"
        tools = [globals().get(t) for t in tool_names if globals().get(t)]
        self.tools = list(self.tools or []) + tools
    
    def add_vars_and_tools(self, var_names:Union[list,str]=None, tool_names:Union[list,str]=None):
        "Add both variables and tools to the chat's lists"
        self.add_tools(tool_names)
        self.add_vars(var_names)
                
c = BackupChat

In [ ]:
md_heir_md = read_url("https://raw.githubusercontent.com/AnswerDotAI/toolslm/refs/heads/main/04_md_hier.ipynb")


In [ ]:
bc = c()

Please try again by using e.g. `bc = dhb.c('model_name')` with a model name in:
azure/eu/gpt-5-2025-08-07
azure/eu/gpt-5-mini-2025-08-07
azure/eu/gpt-5.1
azure/eu/gpt-5.1-chat
azure/eu/gpt-5.1-codex
azure/eu/gpt-5.1-codex-mini
azure/eu/gpt-5-nano-2025-08-07
azure/global/gpt-5.1
azure/global/gpt-5.1-chat
azure/global/gpt-5.1-codex
azure/global/gpt-5.1-codex-mini
azure/gpt-5.1-2025-11-13
azure/gpt-5.1-chat-2025-11-13
azure/gpt-5.1-codex-2025-11-13
azure/gpt-5.1-codex-mini-2025-11-13
azure/gpt-5
azure/gpt-5-2025-08-07
azure/gpt-5-chat
azure/gpt-5-chat-latest
azure/gpt-5-codex
azure/gpt-5-mini
azure/gpt-5-mini-2025-08-07
azure/gpt-5-nano
azure/gpt-5-nano-2025-08-07
azure/gpt-5-pro
azure/gpt-5.1
azure/gpt-5.1-chat
azure/gpt-5.1-codex
azure/gpt-5.1-codex-mini
azure/us/gpt-5-2025-08-07
azure/us/gpt-5-mini-2025-08-07
azure/us/gpt-5-nano-2025-08-07
azure/us/gpt-5.1
azure/us/gpt-5.1-chat
azure/us/gpt-5.1-codex
azure/us/gpt-5.1-codex-mini
gpt-5
gpt-5.1
gpt-5.1-2025-11-13
gpt-5.1-chat-latest
gpt-5

In [ ]:
lisette_md = read_url("https://lisette.answer.ai/")
lisette_md[0:10]

'[ lisette '

In [ ]:
bc = c("openrouter/openai/gpt-5-codex")

The following gets commented out when run (uncommented now so you can run in a test)

In [ ]:
bc("Can you please teach me about Lisette? Only use the info in $`lisette_md`.")

What does the variable `lisette_md` currently contain? If you’d like me to use it directly, could you make it available to the chat (e.g., by running `bc.add_vars('lisette_md')`) and then let me know it’s ready?

<details>

- id: `gen-1763994133-EjtE76wyzTE3j4Ocy3ND`
- model: `openai/gpt-5-codex`
- finish_reason: `stop`
- usage: `Usage(completion_tokens=317, prompt_tokens=3319, total_tokens=3636, completion_tokens_details=CompletionTokensDetailsWrapper(accepted_prediction_tokens=None, audio_tokens=None, reasoning_tokens=256, rejected_prediction_tokens=None, text_tokens=None, image_tokens=0), prompt_tokens_details=PromptTokensDetailsWrapper(audio_tokens=0, cached_tokens=0, text_tokens=None, image_tokens=None, video_tokens=0), cost=0.00731875, is_byok=False, cost_details={'upstream_inference_cost': None, 'upstream_inference_prompt_cost': 0.00414875, 'upstream_inference_completions_cost': 0.00317})`

</details>

**Prompt (openrouter/openai/gpt-5-codex):** Can you please teach me about Lisette? Only use the info in $`lisette_md`.

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

What does the variable `lisette_md` currently contain? If you’d like me to use it directly, could you make it available to the chat (e.g., by running `bc.add_vars('lisette_md')`) and then let me know it’s ready?

In [ ]:
bc.add_vars('lisette_md')
bc("Can you tell me about the library now, based only on the variable, elevator pitch plus example code from the source. I know you are being Socratic but please give answers and not questions on this one.")

Lisette is a higher-level wrapper around the LiteLLM Python SDK that smooths out the rough edges of working with many different LLM providers. LiteLLM already unifies 100+ models behind an OpenAI-style API, but it stays fairly low level: you juggle your own histories, format multi-modal prompts, wire search and tool hookups, and so on. Lisette automates almost all of that while still letting you keep fine-grained control when you need it. You get a drop-in `Chat` class for stateful conversations across any LiteLLM-supported model, simple helpers for mixing text, images, and multiple turns, one-line streaming and async, as well as automatic tool execution and optional provider web-search integration (with citations for models that support them). Prompt caching is built in for the models that can use it, and you only need to supply the relevant API keys as environment variables for whatever providers you want to reach.

A minimal workflow looks like this:

```python
from lisette import *

models = ['claude-sonnet-4-20250514', 'gemini/gemini-2.5-flash', 'openai/gpt-4o']

for model in models:
    chat = Chat(model)
    res = chat("Please tell me about yourself in one brief sentence.")
    display(res)
```

The same interface works no matter which provider you point at, and the library takes care of all provider-specific quirks. From there you can pass multi-message lists or image bytes, prefill responses (for supported providers), register tools with Python type hints, enable search with a single flag, or switch to `AsyncChat` for concurrent apps and combine it with streaming plus tooluse.

<details>

- id: `gen-1763994241-u5E4tKlnoLZtCNsJ1CWt`
- model: `openai/gpt-5-codex`
- finish_reason: `stop`
- usage: `Usage(completion_tokens=422, prompt_tokens=10578, total_tokens=11000, completion_tokens_details=CompletionTokensDetailsWrapper(accepted_prediction_tokens=None, audio_tokens=None, reasoning_tokens=64, rejected_prediction_tokens=None, text_tokens=None, image_tokens=0), prompt_tokens_details=PromptTokensDetailsWrapper(audio_tokens=0, cached_tokens=0, text_tokens=None, image_tokens=None, video_tokens=0), cost=0.0174425, is_byok=False, cost_details={'upstream_inference_cost': None, 'upstream_inference_prompt_cost': 0.0132225, 'upstream_inference_completions_cost': 0.00422})`

</details>

**Prompt (openrouter/openai/gpt-5-codex):** Can you tell me about the library now, based only on the variable, elevator pitch plus example code from the source. I know you are being Socratic but please give answers and not questions on this one.

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

Lisette is a higher-level wrapper around the LiteLLM Python SDK that smooths out the rough edges of working with many different LLM providers. LiteLLM already unifies 100+ models behind an OpenAI-style API, but it stays fairly low level: you juggle your own histories, format multi-modal prompts, wire search and tool hookups, and so on. Lisette automates almost all of that while still letting you keep fine-grained control when you need it. You get a drop-in `Chat` class for stateful conversations across any LiteLLM-supported model, simple helpers for mixing text, images, and multiple turns, one-line streaming and async, as well as automatic tool execution and optional provider web-search integration (with citations for models that support them). Prompt caching is built in for the models that can use it, and you only need to supply the relevant API keys as environment variables for whatever providers you want to reach.

A minimal workflow looks like this:

```python
from lisette import *

models = ['claude-sonnet-4-20250514', 'gemini/gemini-2.5-flash', 'openai/gpt-4o']

for model in models:
    chat = Chat(model)
    res = chat("Please tell me about yourself in one brief sentence.")
    display(res)
```

The same interface works no matter which provider you point at, and the library takes care of all provider-specific quirks. From there you can pass multi-message lists or image bytes, prefill responses (for supported providers), register tools with Python type hints, enable search with a single flag, or switch to `AsyncChat` for concurrent apps and combine it with streaming plus tooluse.

In [ ]:
bc("Can you use tools? For example can you read https://llmstxt.org/index.md and tell me about it? Fetch it, don't store it, give the elevator pitch please.")

Here’s a quick elevator pitch based on what I just read:

**llms.txt** is a simple markdown convention for websites to expose LLM-friendly documentation in one predictable place. By keeping `/llms.txt` concise—an H1 title, a short summary, and curated links to clean `.md` versions of important pages—it sidesteps the noise of full HTML sites and makes it easy for language models (and their tooling) to grab high-value context. It complements, rather than replaces, things like sitemaps or robots.txt, and comes with tooling (CLI, plugins, etc.) to help teams expand the spec into richer context bundles.

<details>

- id: `gen-1763994283-CmXtYOkchauXvnyPigVQ`
- model: `openai/gpt-5-codex`
- finish_reason: `stop`
- usage: `Usage(completion_tokens=137, prompt_tokens=16189, total_tokens=16326, completion_tokens_details=CompletionTokensDetailsWrapper(accepted_prediction_tokens=None, audio_tokens=None, reasoning_tokens=0, rejected_prediction_tokens=None, text_tokens=None, image_tokens=0), prompt_tokens_details=PromptTokensDetailsWrapper(audio_tokens=0, cached_tokens=0, text_tokens=None, image_tokens=None, video_tokens=0), cost=0.02160625, is_byok=False, cost_details={'upstream_inference_cost': None, 'upstream_inference_prompt_cost': 0.02023625, 'upstream_inference_completions_cost': 0.00137})`

</details>

`llms.txt` is a proposal for every website to publish a concise, LLM-friendly Markdown companion at `/llms.txt`. It gives models a curated overview, links to clean `.md` versions of key pages, and follows a predictable structure (H1 title, short summary, optional detail, then labeled link sections). The goal is to make it easy for language models to grab high-signal context without wrangling messy HTML, while staying compatible with existing web standards like `robots.txt` and sitemaps.

<details>

- id: `gen-1763994333-LwPkgTQX9ikegpC1NQti`
- model: `openai/gpt-5-codex`
- finish_reason: `stop`
- usage: `Usage(completion_tokens=240, prompt_tokens=16189, total_tokens=16429, completion_tokens_details=CompletionTokensDetailsWrapper(accepted_prediction_tokens=None, audio_tokens=None, reasoning_tokens=128, rejected_prediction_tokens=None, text_tokens=None, image_tokens=0), prompt_tokens_details=PromptTokensDetailsWrapper(audio_tokens=0, cached_tokens=13440, text_tokens=None, image_tokens=None, video_tokens=0), cost=0.00751625, is_byok=False, cost_details={'upstream_inference_cost': None, 'upstream_inference_prompt_cost': 0.00511625, 'upstream_inference_completions_cost': 0.0024})`

</details>

**Prompt (openrouter/openai/gpt-5-codex):** Can you use tools? For example can you read https://llmstxt.org/index.md and tell me about it? Fetch it, don't store it, give the elevator pitch please.

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

<details class='tool-usage-details'>

```json
{
  "id": "call_lAFPpDr5RciSYwMGKlzHua8Y",
  "call": {
    "function": "read_url",
    "arguments": {
      "url": "https://llmstxt.org/index.md",
      "as_md": true,
      "extract_section": false,
      "selector": ""
    }
  },
  "result": "# The /llms.txt file\nJeremy Howard\n2024-09-03\n\n## Background\n\nLarge language models increasingly rel<TRUNCATED>"
}
```

</details>

`llms.txt` is a proposal for every website to publish a concise, LLM-friendly Markdown companion at `/llms.txt`. It gives models a curated overview, links to clean `.md` versions of key pages, and follows a predictable structure (H1 title, short summary, optional detail, then labeled link sections). The goal is to make it easy for language models to grab high-signal context without wrangling messy HTML, while staying compatible with existing web standards like `robots.txt` and sitemaps.

In [ ]:
bc("Now use your tool and summarize https://raw.githubusercontent.com/AnswerDotAI/fhdaisy/refs/heads/main/README.md please - give a code example")

`fhdaisy` is a thin Python layer that brings DaisyUI’s Tailwind-based component set directly into FastHTML apps. Instead of sprinkling HTML strings like `<button class="btn btn-primary">`, you work with Python-first components (`Btn`, `Card`, `Alert`, etc.) that automatically apply the right DaisyUI classes and underlying HTML tags. Modifiers are shortened (`-primary`, `-outline`, `-sm`), yet expand to the full DaisyUI class list under the hood, keeping your UI code concise and readable. Extras in `fhdaisy.xtras` offer ready-made helpers for recurring UI patterns (accordions, forms, …), while still letting you compose or extend components as needed.

Typical flow:

```python
pip install fhdaisy
```

then:

```python
from fasthtml.common import *
from fhdaisy import *

page = Div(
    Card(
        CardBody(
            H2("Welcome to FastHTML + DaisyUI"),
            P("This card is built from Python components, not raw HTML."),
            Btn("Click me", cls='-primary -sm')
        ),
        cls='-compact -shadow'
    )
)

print(page)
```

The snippet yields well-structured HTML where each element already carries the expected DaisyUI classes, so you can focus on layout and behavior rather than CSS glue code. For a deeper dive—including advanced components, helper utilities, and live demos—check the project’s documentation site linked from the README.

<details>

- id: `gen-1763994357-VoP1BBTW593bbYjKHcTL`
- model: `openai/gpt-5-codex`
- finish_reason: `stop`
- usage: `Usage(completion_tokens=631, prompt_tokens=18980, total_tokens=19611, completion_tokens_details=CompletionTokensDetailsWrapper(accepted_prediction_tokens=None, audio_tokens=None, reasoning_tokens=320, rejected_prediction_tokens=None, text_tokens=None, image_tokens=0), prompt_tokens_details=PromptTokensDetailsWrapper(audio_tokens=0, cached_tokens=17664, text_tokens=None, image_tokens=None, video_tokens=0), cost=0.010163, is_byok=False, cost_details={'upstream_inference_cost': None, 'upstream_inference_prompt_cost': 0.003853, 'upstream_inference_completions_cost': 0.00631})`

</details>

**Prompt (openrouter/openai/gpt-5-codex):** Now use your tool and summarize https://raw.githubusercontent.com/AnswerDotAI/fhdaisy/refs/heads/main/README.md please - give a code example

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

<details class='tool-usage-details'>

```json
{
  "id": "call_7FvqavATiA7YIfVsneBjiPQH",
  "call": {
    "function": "read_url",
    "arguments": {
      "url": "https://raw.githubusercontent.com/AnswerDotAI/fhdaisy/refs/heads/main/README.md",
      "as_md": true,
      "extract_section": false,
      "selector": ""
    }
  },
  "result": "# fhdaisy\n\n`fhdaisy` is a Python wrapper for [DaisyUI](https://daisyui.com/) that\nbrings its compone<TRUNCATED>"
}
```

</details>

`fhdaisy` is a thin Python layer that brings DaisyUI’s Tailwind-based component set directly into FastHTML apps. Instead of sprinkling HTML strings like `<button class="btn btn-primary">`, you work with Python-first components (`Btn`, `Card`, `Alert`, etc.) that automatically apply the right DaisyUI classes and underlying HTML tags. Modifiers are shortened (`-primary`, `-outline`, `-sm`), yet expand to the full DaisyUI class list under the hood, keeping your UI code concise and readable. Extras in `fhdaisy.xtras` offer ready-made helpers for recurring UI patterns (accordions, forms, …), while still letting you compose or extend components as needed.

Typical flow:

```python
pip install fhdaisy
```

then:

```python
from fasthtml.common import *
from fhdaisy import *

page = Div(
    Card(
        CardBody(
            H2("Welcome to FastHTML + DaisyUI"),
            P("This card is built from Python components, not raw HTML."),
            Btn("Click me", cls='-primary -sm')
        ),
        cls='-compact -shadow'
    )
)

print(page)
```

The snippet yields well-structured HTML where each element already carries the expected DaisyUI classes, so you can focus on layout and behavior rather than CSS glue code. For a deeper dive—including advanced components, helper utilities, and live demos—check the project’s documentation site linked from the README.